In [0]:
from tensorflow.keras.datasets import mnist

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [3]:
# Display information about the dataset
print(train_images.shape)
print(len(train_labels))
print(train_labels)

(60000, 28, 28)
60000
[5 0 4 ... 5 6 8]


In [4]:
print(test_images.shape)
print(len(test_labels))
print(test_labels)

(10000, 28, 28)
10000
[7 2 1 ... 4 5 6]


In [6]:
from tensorflow import keras
from tensorflow.keras import layers

# Build a simple neural network model using Keras
model = keras.Sequential([
    layers.Dense(512, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [7]:
# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Preprocess the images
train_images = train_images.reshape(60000, 28 * 28)
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape(10000, 28 * 28)
test_images = test_images.astype('float32') / 255

In [10]:
# Train the model
model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 0s 968us/step - loss: 0.0290 - accuracy: 0.9916
Epoch 2/5
469/469 [==============================] - 1s 1ms/step - loss: 0.0231 - accuracy: 0.9930
Epoch 3/5
469/469 [==============================] - 1s 1ms/step - loss: 0.0169 - accuracy: 0.9952
Epoch 4/5
469/469 [==============================] - 1s 1ms/step - loss: 0.0132 - accuracy: 0.9962
Epoch 5/5
469/469 [==============================] - 1s 1ms/step - loss: 0.0102 - accuracy: 0.9975


In [11]:
# Make predictions on test digits
test_digits = test_images[0:10]
predictions = model.predict(test_digits)
print(predictions[0])

1/1 [==============================] - 0s 48ms/step
[1.8480515e-09 1.7267963e-10 3.7623423e-07 1.3334193e-05 1.4909249e-13
 1.1506768e-09 8.0809442e-15 9.9998546e-01 5.1674420e-09 7.9232262e-07]


In [12]:
predictions[0].argmax()

7

In [13]:
test_labels[0]

7

In [14]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_loss, test_acc)

313/313 [==============================] - 0s 735us/step - loss: 0.0628 - accuracy: 0.9819
0.06280253082513809 0.9818999767303467


In [15]:
import tensorflow as tf

# Implement a simple neural network from scratch
class NaiveDense:
        # Class to represent a dense layer
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        # Initialize weights and bias
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0., maxval=1e-1)
        self.W = tf.Variable(w_initial_value)
        
        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)
    
    def __call__(self, inputs):
        # Forward pass through the layer
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self):
        # Return weights and bias
        return [self.W, self.b]

In [18]:
class NaiveSequential:
    # Class to represent a sequential model
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, inputs):
        # Forward pass through the model
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        # Return weights of all layers
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [19]:
# Create a simple sequential model using the naive implementation
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [20]:
import math

# Implement training and batch generation
class BatchGenerator:
    # Class to generate batches of images and labels
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
    
    def next(self):
        # Return the next batch of images and labels
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [21]:
# Update weights using gradient descent
def update_weights(gradients, weights, learning_rate=1e-3):
    for g, w in zip(gradients, weights):
        w.assign(g * learning_rate)

# Perform a single training step
def train_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [22]:
from tensorflow.keras import optimizers

# Define the optimizer and its update function
optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [31]:
# Define a function to train the model
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print("Epoch {}/{}".format(epoch_counter + 1, epochs))
        batch_generator = BatchGenerator(images, labels, batch_size)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = train_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"Batch {batch_counter} loss: {loss:.2f}")

In [ ]:
# Train the naive model
fit(model, train_images, train_labels, epochs=10, batch_size=128)

In [37]:
import numpy as np

# Evaluate the naive model on the test set
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"Accuracy: {matches.mean():.2f}")

Accuracy: 0.87
